a function to calculate word error rate

In [ ]:
def wer1(ref, hyp ,debug=False):
    ref=ref.lower()
    hyp=hyp.lower()
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY=1 # Tact
    INS_PENALTY=1 # Tact
    SUB_PENALTY=1 # Tact
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append(".\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = list(reversed(lines))
        for i,line in enumerate(lines):
            if(line.startswith('.')):
                if(i-1 > 0):
                    if(not lines[i-1].startswith('.')):
                        print(line)
                        continue
                if(i+1 < len(lines)):
                    if(lines[i+1].startswith('.')):continue
            print(line)
        print("Ncor " + str(numCor))
        print("Nsub " + str(numSub))
        print("Ndel " + str(numDel))
        print("Nins " + str(numIns))
    return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'Cor':numCor, 'Sub':numSub, 'Ins':numIns, 'Del':numDel}

function to normalize our string lemmatization, case converison

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
def lemmatized(para):
    para=para.lower()
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)

    lemmatizer = WordNetLemmatizer()
    final=''
    sent_text = nltk.sent_tokenize(para) 
    for sentence in sent_text:
        x=([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])
        x=" ".join(x)+' '
        final=final+x
    return(final)

Calling the azure speech services , enter your subscription key

In [ ]:
import time
import wave
import azure.cognitiveservices.speech as speechsdk

# Set up the subscription info for the Speech Service:
# Replace with your own subscription key and service region (e.g., "westus").

speech_key, service_region = "subscription_key", "westus"

def speech_recognize_continuous_from_file(weatherfilename):
    """performs continuous speech recognition with input from an audio file"""
    # <SpeechContinuousRecognitionWithFile>
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region,speech_recognition_language='en-us')
    audio_config = speechsdk.audio.AudioConfig(filename=weatherfilename)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt):
        """callback that stops continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True
    
    speech_recognizer.recognized.connect(lambda evt: obj1.append(evt) )
#     # Connect callbacks to the events fired by the speech recognizer , comment out these if you don't want status updates
    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)
    # </SpeechContinuousRecognitionWithFile>


Testing the audios

In [ ]:
import re
from jiwer import wer
import numpy as np
transcripts=[]
#enter path for audio files to be tested
path='C:\\Users\\eklavya\\istem\\test\\indian\\New Folder\\'
#enter name of audio files to be tested
vids=['Kiran_Bedi_How_I_remade_one_of_Indias_toughest_prisonst0',
      'A_well_educated_mind_vs_a_well_formed_mind_Dr_Shashi_Tharoor_at_TEDxGateway_2013t1',
      "Thoughts_on_humanity_fame_and_love_Shah_Rukh_Khant1",
      "What_makes_life_complete_Gaur_Gopal_Das_TEDxMITPt4",
      "Why_is_India_so_filthy_The_Ugly_Indian_TEDxBangaloret3" 
     ]
for vid in vids:
    print(vid)
    obj1=[]
    speech_recognize_continuous_from_file(path+vid+'.wav')
    st=''
    for i in obj1:
        st=st+' '+((i.result).text)
    transcripts.append(st)

analysing the results word error rate - removing punctuation marks and lemmatizatizing

In [ ]:
import os
ground_truths=[]
for file in vids:
    f= open((file[:-2]+'_transcribed.txt'),'r')
    w=f.readlines()
    ground_truths.append(w[int(file[-1])])
wers=[]

for i in range(len(transcripts)):
    ground=(ground_truths[i].split('\t')[1])
    print('\n',(ground),'\n') # printing the actual ground truth without any normalization
    print('\n',(transcripts[i]),'\n') # printing the actual transcripts without any normalization
    ground=lemmatized(ground)
    ground=" ".join(re.findall("[a-zA-Z0-9]+", ground)) # normalizing - removing punctuations
    ele=" ".join(re.findall("[a-zA-Z0-9]+",(lemmatized(transcripts[i])))) 
    wer1(ground,ele,True)#  comment this line if you only want the word error rate and not the debugging
    wers.append(wer(ground,ele))

In [ ]:
for i in range(len(vids)):
    print(vids[i],'\t',wers[i])